In [1]:
%cd data

/home/workspace/data


In [1]:
!ls

data	etl.py	   song_data	    time_table		Untitled.ipynb
dl.cfg	README.md  spark-warehouse  time_table.parquet


In [11]:
!unzip song-data.zip

Archive:  song-data.zip
   creating: song_data/
  inflating: song_data/.DS_Store     
   creating: song_data/A/
  inflating: song_data/A/.DS_Store   
   creating: song_data/A/A/
  inflating: song_data/A/A/.DS_Store  
   creating: song_data/A/A/A/
  inflating: song_data/A/A/A/TRAAAAW128F429D538.json  
  inflating: song_data/A/A/A/TRAAABD128F429CF47.json  
  inflating: song_data/A/A/A/TRAAADZ128F9348C2E.json  
  inflating: song_data/A/A/A/TRAAAEF128F4273421.json  
  inflating: song_data/A/A/A/TRAAAFD128F92F423A.json  
  inflating: song_data/A/A/A/TRAAAMO128F1481E7F.json  
  inflating: song_data/A/A/A/TRAAAMQ128F1460CD3.json  
  inflating: song_data/A/A/A/TRAAAPK128E0786D96.json  
  inflating: song_data/A/A/A/TRAAARJ128F9320760.json  
  inflating: song_data/A/A/A/TRAAAVG12903CFA543.json  
  inflating: song_data/A/A/A/TRAAAVO128F93133D4.json  
   creating: song_data/A/A/B/
  inflating: song_data/A/A/B/TRAABCL128F4286650.json  
  inflating: song_data/A/A/B/TRAABDL12903CAABBA.json  
  inflat

In [13]:
%mkdir log_data
!unzip log-data.zip -d log_data

mkdir: cannot create directory ‘log_data’: File exists
Archive:  log-data.zip
  inflating: log_data/2018-11-01-events.json  
  inflating: log_data/2018-11-02-events.json  
  inflating: log_data/2018-11-03-events.json  
  inflating: log_data/2018-11-04-events.json  
  inflating: log_data/2018-11-05-events.json  
  inflating: log_data/2018-11-06-events.json  
  inflating: log_data/2018-11-07-events.json  
  inflating: log_data/2018-11-08-events.json  
  inflating: log_data/2018-11-09-events.json  
  inflating: log_data/2018-11-10-events.json  
  inflating: log_data/2018-11-11-events.json  
  inflating: log_data/2018-11-12-events.json  
  inflating: log_data/2018-11-13-events.json  
  inflating: log_data/2018-11-14-events.json  
  inflating: log_data/2018-11-15-events.json  
  inflating: log_data/2018-11-16-events.json  
  inflating: log_data/2018-11-17-events.json  
  inflating: log_data/2018-11-18-events.json  
  inflating: log_data/2018-11-19-events.json  
  inflating: log_data/2018-11

In [22]:
import configparser
from datetime import datetime
import os
import glob 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, monotonically_increasing_id
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, FloatType, LongType
# from pyspark.sql.types.DataType import StringType, IntegerType, DoubleType

In [3]:
# def create_spark_session():
#     spark = SparkSession \
#         .builder \
#         .getOrCreate()
#     return spark

spark = SparkSession.builder.getOrCreate()

In [4]:
song_data = glob.glob('/home/workspace/data/song_data/*/*/*/*.json')

In [18]:
# data structure 
schema = StructType() \
  .add("artist_id",StringType(),False) \
  .add("artist_latitude",FloatType(),True) \
  .add("artist_location",StringType(),True) \
  .add("artist_longitude",FloatType(),True) \
  .add("artist_name",StringType(),True) \
  .add("duration",FloatType(),True) \
  .add("num_songs",IntegerType(),True) \
  .add("song_id",StringType(),False) \
  .add("title",StringType(),True) \
  .add("year",IntegerType(),True) 

In [19]:
song_df = spark.read.schema(schema).json(song_data)
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: float (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: float (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)



In [16]:
song_df = spark.read.json(song_data)
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [7]:
song_df.show()

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           null|                    |            null|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
|AREBBGV1187FB523D2|           null|         Houston, TX|            null|Mike Jones (Featu...|173.66159|        1|SOOLYAZ12A6701F4A6|Laws Patrolling (...|   0|
|ARMAC4T1187FB3FA4C|       40.82624|   Morris Plains, NJ|       -74.47995|The Dillinger Esc...|207.77751|        1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
|ARPBNLO1187FB3D52F|       40.7145

In [26]:
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']

In [27]:
songs_table = song_df.selectExpr(*song_columns).dropDuplicates(subset=['song_id'])
songs_table.count()

71

In [16]:
songs_table.groupBy('artist_id').count().orderBy("count").show()

+------------------+-----+
|         artist_id|count|
+------------------+-----+
|AR8ZCNI1187B9A069B|    1|
|AR9AWNF1187B9AB0B4|    1|
|AR0RCMP1187FB3F427|    1|
|AR0IAWL1187B9A96D0|    1|
|ARI3BMM1187FB4255E|    1|
|AR7SMBG1187B9B9066|    1|
|ARMAC4T1187FB3FA4C|    1|
|AREDL271187FB40F44|    1|
|ARKRRTF1187B9984DA|    1|
|AR051KA1187B98B2FF|    1|
|AR10USD1187B99F3F1|    1|
|AR7ZKHQ1187B98DD73|    1|
|ARQ9BO41187FB5CF1F|    1|
|AR7G5I41187FB4CE6C|    1|
|ARGCY1Y1187B9A4FA5|    1|
|ARAJPHH1187FB5566A|    1|
|AR558FS1187FB45658|    1|
|ARKFYS91187B98E58F|    1|
|ARD842G1187B997376|    1|
|ARJIE2Y1187B994AB7|    1|
+------------------+-----+
only showing top 20 rows



In [35]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artists_table = song_df.selectExpr(*artist_columns)
artists_table.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARDR4AC1187FB371A1|Montserrat Caball...|                    |           null|            null|
|AREBBGV1187FB523D2|Mike Jones (Featu...|         Houston, TX|           null|            null|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|   Morris Plains, NJ|       40.82624|       -74.47995|
|ARPBNLO1187FB3D52F|            Tiny Tim|        New York, NY|       40.71455|       -74.00712|
|ARDNS031187B9924F0|          Tim Wilson|             Georgia|       32.67828|       -83.22295|
|ARNF6401187FB57032|   Sophie B. Hawkins|New York, NY [Man...|       40.79086|       -73.96644|
|ARLTWXK1187FB5A3F8|         King Curtis|      Fort Worth, TX|       32.74863|       -97.32925|
|ARPFHN61187FB575F6|         Lupe Fiasco

In [23]:
songs_table.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
|SONWXQJ12A8C134D94|The Ballad Of Sle...|ARNF6401187FB57032|1994|  305.162|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|   0|326.00771|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|   0|279.97995|
|SODUJBS12A8C132150|Wessex Loses a Bride|ARI2JSK1187FB496EF|   0|111.62077|
|SOGXHEG12AB018653E|It Makes No Diffe...|AR0RCMP1187FB3F427|1992|133.32853|
|SOBZBAZ12A6

In [25]:
songs_table.write.option("header",True) \
        .partitionBy("artist_id","year") \
        .mode("overwrite") \
        .csv("song_table.csv")

In [30]:
artist_columns = ['artist_id', 'artist_name as name', 'artist_location as location', 'artist_latitude as latitude', 'artist_longitude as longitude']
artists_table = song_df.selectExpr(*artist_columns).dropDuplicates(subset=['artist_id'])
artists_table.show()

+------------------+--------------------+--------------------+--------+---------+
|         artist_id|                name|            location|latitude|longitude|
+------------------+--------------------+--------------------+--------+---------+
|AR9AWNF1187B9AB0B4|Kenny G featuring...|Seattle, Washingt...|    null|     null|
|AR0IAWL1187B9A96D0|        Danilo Perez|              Panama|  8.4177|-80.11278|
|AR0RCMP1187FB3F427|    Billie Jo Spears|        Beaumont, TX|30.08615|-94.10158|
|AREDL271187FB40F44|        Soul Mekanik|                    |    null|     null|
|ARI3BMM1187FB4255E|        Alice Stuart|          Washington| 38.8991|  -77.029|
|AR7SMBG1187B9B9066|         Los Manolos|                    |    null|     null|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|   Morris Plains, NJ|40.82624|-74.47995|
|ARNTLGG11E2835DDB9|                 Clp|                    |    null|     null|
|ARKRRTF1187B9984DA|    Sonora Santanera|                    |    null|     null|
|AR051KA1187B98B

In [20]:
log_data = glob.glob('/home/workspace/data/log_data/*.json')
log_data

['/home/workspace/data/log_data/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018-11-14-events.json',
 '/home/workspace/data/log_data/2018-11-27-events.json',
 '/home/workspace/data/log_data/2018-11-21-events.json',
 '/home/workspace/data/log_data

In [23]:
schema = StructType() \
      .add("artist",StringType(),True) \
      .add("auth",StringType(),True) \
      .add("firstName",StringType(),True) \
      .add("gender",StringType(),True) \
      .add("itemInSession",IntegerType(),True) \
      .add("lastName",StringType(),True) \
      .add("length",FloatType(),True) \
      .add("level",StringType(),True) \
      .add("location",StringType(),True) \
      .add("method",StringType(),True) \
      .add("page",StringType(),True) \
      .add("registration",StringType(),True) \
      .add("sessionId",IntegerType(),True) \
      .add("song",StringType(),True) \
      .add("status",IntegerType(),True) \
      .add("ts",LongType(),True) \
      .add("userAgent",StringType(),True) \
      .add("userId",StringType(),True) 

In [24]:
df = spark.read.schema(schema).json(log_data)
df.count()

8056

In [6]:
df = df.where(df.page=='NextSong')
df.count()

6820

In [7]:
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
users_table = df.selectExpr(*user_columns)
users_table.show()

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    49|    Chloe|  Cuevas|     F| paid|
+------+---------+--------+------+-----+
only showing top

In [8]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|         The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyva

In [9]:
df2 = df.withColumn("time", (col("ts")/1000).cast("timestamp"))
df2.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|                time|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:...|
|         The Prodig

In [10]:
df2.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|                time|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:...|
|         The Prodig

In [20]:
# import pyspark.sql.functions as F
# time_column = ['year', 'month', 'dayofmonth', 'hour', 'weekofyear']
# get_datetime = udf(lambda x: col('time').cast(x))
df3 = df2.withColumn("year", year(col("time")))\
        .withColumn("month", month(col("time")))\
        .withColumn("dayofmonth", dayofmonth(col("time")))\
        .withColumn("hour", hour(col("time")))\
        .withColumn("weekofyear", weekofyear(col("time")))\
        .withColumn("weekday", date_format(col("time"), 'u'))



df3.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+----+-----+----------+----+----------+-------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|                time|year|month|dayofmonth|hour|weekofyear|weekday|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+----+-----+----------+----+----------+-------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.

In [21]:
time_column = ['ts', 'hour', 'dayofmonth as day', 'weekofyear as week', 'month', 'year', 'weekday']
time_table = df3.selectExpr(*time_column)

In [22]:
time_table.show()

+-------------+----+---+----+-----+----+-------+
|           ts|hour|day|week|month|year|weekday|
+-------------+----+---+----+-----+----+-------+
|1542241826796|   0| 15|  46|   11|2018|      4|
|1542242481796|   0| 15|  46|   11|2018|      4|
|1542242741796|   0| 15|  46|   11|2018|      4|
|1542253449796|   3| 15|  46|   11|2018|      4|
|1542260935796|   5| 15|  46|   11|2018|      4|
|1542261224796|   5| 15|  46|   11|2018|      4|
|1542261356796|   5| 15|  46|   11|2018|      4|
|1542261662796|   6| 15|  46|   11|2018|      4|
|1542262057796|   6| 15|  46|   11|2018|      4|
|1542262233796|   6| 15|  46|   11|2018|      4|
|1542262434796|   6| 15|  46|   11|2018|      4|
|1542262456796|   6| 15|  46|   11|2018|      4|
|1542262679796|   6| 15|  46|   11|2018|      4|
|1542262728796|   6| 15|  46|   11|2018|      4|
|1542262893796|   6| 15|  46|   11|2018|      4|
|1542263158796|   6| 15|  46|   11|2018|      4|
|1542263378796|   6| 15|  46|   11|2018|      4|
|1542265716796|   7|

In [24]:
time_table.write.partitionBy("year", "month").parquet('time_table')

In [25]:
df4 = spark.read.parquet("time_table")

In [26]:
df4.show()

+-------------+----+---+----+-------+----+-----+
|           ts|hour|day|week|weekday|year|month|
+-------------+----+---+----+-------+----+-----+
|1542241826796|   0| 15|  46|      4|2018|   11|
|1542242481796|   0| 15|  46|      4|2018|   11|
|1542242741796|   0| 15|  46|      4|2018|   11|
|1542253449796|   3| 15|  46|      4|2018|   11|
|1542260935796|   5| 15|  46|      4|2018|   11|
|1542261224796|   5| 15|  46|      4|2018|   11|
|1542261356796|   5| 15|  46|      4|2018|   11|
|1542261662796|   6| 15|  46|      4|2018|   11|
|1542262057796|   6| 15|  46|      4|2018|   11|
|1542262233796|   6| 15|  46|      4|2018|   11|
|1542262434796|   6| 15|  46|      4|2018|   11|
|1542262456796|   6| 15|  46|      4|2018|   11|
|1542262679796|   6| 15|  46|      4|2018|   11|
|1542262728796|   6| 15|  46|      4|2018|   11|
|1542262893796|   6| 15|  46|      4|2018|   11|
|1542263158796|   6| 15|  46|      4|2018|   11|
|1542263378796|   6| 15|  46|      4|2018|   11|
|1542265716796|   7|

In [33]:
songs_table.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
|SONWXQJ12A8C134D94|The Ballad Of Sle...|ARNF6401187FB57032|1994|  305.162|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|   0|326.00771|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|   0|279.97995|
|SODUJBS12A8C132150|Wessex Loses a Bride|ARI2JSK1187FB496EF|   0|111.62077|
|SOGXHEG12AB018653E|It Makes No Diffe...|AR0RCMP1187FB3F427|1992|133.32853|
|SOBZBAZ12A6

In [37]:
artists_table.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARDR4AC1187FB371A1|Montserrat Caball...|                    |           null|            null|
|AREBBGV1187FB523D2|Mike Jones (Featu...|         Houston, TX|           null|            null|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|   Morris Plains, NJ|       40.82624|       -74.47995|
|ARPBNLO1187FB3D52F|            Tiny Tim|        New York, NY|       40.71455|       -74.00712|
|ARDNS031187B9924F0|          Tim Wilson|             Georgia|       32.67828|       -83.22295|
|ARNF6401187FB57032|   Sophie B. Hawkins|New York, NY [Man...|       40.79086|       -73.96644|
|ARLTWXK1187FB5A3F8|         King Curtis|      Fort Worth, TX|       32.74863|       -97.32925|
|ARPFHN61187FB575F6|         Lupe Fiasco

In [38]:
song_artist_table = songs_table.join(artists_table, songs_table.artist_id == artists_table.artist_id)

In [39]:
song_artist_table.show()

+------------------+--------------------+------------------+----+---------+------------------+--------------------+--------------------+---------------+----------------+
|           song_id|               title|         artist_id|year| duration|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+------------------+----+---------+------------------+--------------------+--------------------+---------------+----------------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|ARDR4AC1187FB371A1|Montserrat Caball...|                    |           null|            null|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|AREBBGV1187FB523D2|Mike Jones (Featu...|         Houston, TX|           null|            null|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|ARMAC4T1187FB3FA4C|The Dillinger Esc...|   Morris Plains, NJ|       40.8262

In [41]:
df5 = df3.join(songs_table, df3.song == songs_table.title)

In [42]:
df5.show()

+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------+------+-------------+--------------------+------+--------------------+----+-----+----------+----+----------+-------+------------------+--------------+------------------+----+---------+
|           artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|          song|status|           ts|           userAgent|userId|                time|year|month|dayofmonth|hour|weekofyear|weekday|           song_id|         title|         artist_id|year| duration|
+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------+------+-------------+--------------------+------+--------------------+----+-----+----------+----+----------+-------+------------------+

In [43]:
songs_table.select(['song_id', 'title', 'artist_id']).show()

+------------------+--------------------+------------------+
|           song_id|               title|         artist_id|
+------------------+--------------------+------------------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|
|SONWXQJ12A8C134D94|The Ballad Of Sle...|ARNF6401187FB57032|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|
|SODUJBS12A8C132150|Wessex Loses a Bride|ARI2JSK1187FB496EF|
|SOGXHEG12AB018653E|It Makes No Diffe...|AR0RCMP1187FB3F427|
|SOBZBAZ12A6D4F8742|      Spanish Grease|AROUOZZ1187B9ABE51|
|SOZHPGD12A8C1394FE|     Baby Come To Me|AR9AWNF1187B9AB0B4|
|SOBLGCN12AB0183212|James (Hold The L...|AR47JEX1187B995D81|
|SOYMRWW12A6D4FAB14|The 

In [47]:
songs_table = songs_table.select(['song_id', 'title', 'artist_id'])

In [48]:
songs_table.show()

+------------------+--------------------+------------------+
|           song_id|               title|         artist_id|
+------------------+--------------------+------------------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|
|SONWXQJ12A8C134D94|The Ballad Of Sle...|ARNF6401187FB57032|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|
|SODUJBS12A8C132150|Wessex Loses a Bride|ARI2JSK1187FB496EF|
|SOGXHEG12AB018653E|It Makes No Diffe...|AR0RCMP1187FB3F427|
|SOBZBAZ12A6D4F8742|      Spanish Grease|AROUOZZ1187B9ABE51|
|SOZHPGD12A8C1394FE|     Baby Come To Me|AR9AWNF1187B9AB0B4|
|SOBLGCN12AB0183212|James (Hold The L...|AR47JEX1187B995D81|
|SOYMRWW12A6D4FAB14|The 

In [50]:
df3.select(['ts', 'userId', 'level', 'sessionId', 'location', 'userAgent', 'song']).join(songs_table, songs_table.title == df3.song).count()

4

In [13]:
df6 = songs_table.withColumn('id', monotonically_increasing_id())
df6.show()

+------------------+--------------------+------------------+----+---------+------------+
|           song_id|               title|         artist_id|year| duration|          id|
+------------------+--------------------+------------------+----+---------+------------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934| 25769803776|
|SOMZWCG12A8C13C480|    I Didn't Mean To|ARD7TVE1187B99BFB1|   0|218.93179| 34359738368|
|SOUPIRU12A6D4FA1E1| Der Kleine Dompfaff|ARJIE2Y1187B994AB7|   0|152.92036| 42949672960|
|SOXVLOJ12AB0189215|     Amor De Cabaret|ARKRRTF1187B9984DA|   0|177.47546| 42949672961|
|SOWTBJW12AC468AC6E|Broken-Down Merry...|ARQGYP71187FB44566|   0|151.84934| 94489280512|
|SOBONFF12A6D4F84D8|Tonight Will Be A...|ARIK43K1187B9AE54C|1986| 307.3824|128849018880|
|SOPVXLX12A8C1402D5|    Larger Than Life|AR3JMC51187B9AE49D|1999|236.25098|171798691840|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|197568495616|
|SOBKWDJ12A8C13B2F3|W

In [39]:
artists_table = artists_table.select('name')

In [40]:
df7 = df.select(['ts', 'userId', 'level', 'sessionId', 'location', 'userAgent', 'song', 'artist']).join(songs_table, songs_table.title == df.song).join(artists_table, artists_table.name == df.artist)

In [41]:
df7.show()

+-------------+------+-----+---------+--------------------+--------------------+--------------+------+------------------+--------------+------------------+----+---------+-----+
|           ts|userId|level|sessionId|            location|           userAgent|          song|artist|           song_id|         title|         artist_id|year| duration| name|
+-------------+------+-----+---------+--------------------+--------------------+--------------+------+------------------+--------------+------------------+----+---------+-----+
|1542837407796|    15| paid|      818|Chicago-Napervill...|"Mozilla/5.0 (X11...|Setanta matins| Elena|SOZCTXZ12AB0182364|Setanta matins|AR5KOSW1187FB35FF4|   0|269.58322|Elena|
+-------------+------+-----+---------+--------------------+--------------------+--------------+------+------------------+--------------+------------------+----+---------+-----+



In [43]:
songplays_column = ['ts as start_time', 'userId as user_id', 'level', 'song_id', 'artist_id', 'sessionId as session_id', 'location', 'userAgent as user_agent']
songplays_table = df7.selectExpr(*songplays_column)

In [44]:
songplays_table.show()

+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|   start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|1542837407796|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+



In [48]:
songplays_table = songplays_table.withColumn('songplay_id', monotonically_increasing_id()) \
                    
songplays_table.show()

+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+
|   start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|songplay_id|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+
|1542837407796|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|          0|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+



In [53]:
songplays_table.select(
    year((col("start_time")/1000).cast("timestamp")).alias('year'), 
    month((col("start_time")/1000).cast("timestamp")).alias('month')).show()  

+----+-----+
|year|month|
+----+-----+
|2018|   11|
+----+-----+



In [54]:
songplays_table.withColumn('year', year((col("start_time")/1000).cast("timestamp"))).show()

+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+----+
|   start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|songplay_id|year|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+----+
|1542837407796|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|          0|2018|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+----+

